#Merge datasets again (Alkon and Reynolds) with the new annotations.

Reynolds will mantain the first annotation before my processing as now Alkon has similar annotation after the celltypist.

In [0]:
.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat_v2/", .libPaths()))

library(Seurat)
library(dplyr)
library(ggplot2)
library(patchwork)
options(future.globals.maxSize = 1e9)


In [0]:
alkon <- readRDS("/dbfs/mnt/sandbox/TFM_PAULA/ALKON_CELLTYPIST_TFM.rds") #new annotation
reynolds <- readRDS("/dbfs/mnt/sandbox/TFM_PAULA/REYNOLDS_PROCESSED_TFM.rds")

In [0]:
alkon$dataset <- "alkon"
reynolds$dataset <- "reynolds"

In [0]:
names(reynolds@meta.data)

In [0]:
reynolds_subset <- subset(reynolds, final_clustering == "Differentiated_KC*")
nrow(reynolds_subset)

In [0]:
reynolds_subset_2 <- subset(reynolds, final_clustering == "Differentiated_KC")
nrow(reynolds_subset)

In [0]:
identical(reynolds_subset_2, reynolds_subset)

Differentiated_KC and Differentiated_KC* are the same cells.

In [0]:
options(repr.plot.width=2200, repr.plot.height=1200)

In [0]:
DimPlot(reynolds, group.by="final_clustering", label= T) + DimPlot(alkon, group.by="celltypist", label= T)

In [0]:
reynolds$celltypist <- reynolds$final_clustering
reynolds$celltypist <- gsub("Undifferentiated_KC.*", "Undifferentiated_KC", reynolds$celltypist)
reynolds$celltypist <- gsub("Differentiated_KC.*|Differentiated_KC", "Differentiated_KC", reynolds$celltypist)
reynolds$celltypist <- gsub("Proliferating_KC", "Prolif", reynolds$celltypist)
reynolds$celltypist <- gsub("F1|F2|F3", "Fibroblasts", reynolds$celltypist)
reynolds$celltypist <- gsub("LC_4|LC_3|LC_1|LC_2", "LC", reynolds$celltypist)
reynolds$celltypist <- gsub("Macro_1|Macro_2", "Macro", reynolds$celltypist)
reynolds$celltypist <- gsub("Pericyte_1_non_inflamm|Pericyte_2_inflamm", "Pericyte", reynolds$celltypist)
reynolds$celltypist <- gsub("VE1|VE2|VE3", "Endothelial blood", reynolds$celltypist)

In [0]:
DimPlot(reynolds, group.by="celltypist", label= T) + DimPlot(alkon, group.by="celltypist", label= T)

The only cells that really matter of this new annotation are the subgroups of Tcells --> Tc and Th, and the keratinocytes differentiated and undifferentiated.

In [0]:
alkon$celltype_AR <- alkon$celltypist
reynolds$celltype_AR <- reynolds$celltypist #To have the same celltype name annotation as the other merged object

In [0]:
unique(alkon$Condition)

In [0]:
alkon$Condition <- ifelse(alkon$Condition == "AD", "Lesional", alkon$Condition)

In [0]:
# # Create a new column 'Condition' with default value 'healthy'
reynolds$Condition <- "HC"

# Update 'Condition' based on the 'Status' and 'Site' columns
reynolds$Condition[reynolds$Status == "Eczema" & reynolds$Site == "lesion"] <- "Lesional"
reynolds$Condition[reynolds$Status == "Eczema" & reynolds$Site == "non_lesion"] <- "Non_lesional"


In [0]:
unique(reynolds$Condition)

In [0]:
alkon$Condition_AR <- alkon$Condition
reynolds$Condition_AR <- reynolds$Condition

In [0]:
reynolds$Sample_id <- reynolds$donor_id

In [0]:
AR <- merge(alkon, y = reynolds, add.cell.ids = c("alkon", "reynolds"))

In [0]:
AR <- NormalizeData(AR)
AR <- FindVariableFeatures(AR)
AR <- ScaleData(AR)
AR <- RunPCA(AR)

In [0]:
AR <- FindNeighbors(AR, dims = 1:30, reduction = "pca")
AR <- FindClusters(AR, resolution = 2, cluster.name = "unintegrated_clusters")

In [0]:
AR <- RunUMAP(AR, dims = 1:30, reduction = "pca", reduction.name = "umap.unintegrated")

In [0]:
DimPlot(AR, group.by="celltype_AR", label= T, reduction = "umap.unintegrated") + DimPlot(AR, group.by="dataset", label= T, reduction = "umap.unintegrated") 

In [0]:
# Save the merged Seurat object
 saveRDS(AR, file="/dbfs/mnt/sandbox/TFM_PAULA/MERGED_ARdatasets_celltypist_TFM.rds")

In [0]:
AR <- readRDS("/dbfs/mnt/sandbox/TFM_PAULA/MERGED_ARdatasets_celltypist_TFM.rds")